In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from openai import OpenAI
from dotenv import load_dotenv
import os
import random
import numpy as np
import regex as re
import pandas as pd

/homes/ngetach/miniconda/envs/project/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_KEY')


In [3]:
lowercases = list('abcdefghijklmnopqrstuvwxyz')
uppercases = list('ABCDEFGHIJKLMNOPQRSTUVWXYZ')

def build_logical_statement(hops, facts, negations=True):
    sample_vars = random.sample(facts, hops)
    if negations:
        sample_vars = ["not " + s if random.choice([True, False]) else s for s in sample_vars]
    output = ''
    for i in range(len(sample_vars)-1):
        generation_str = f'If {sample_vars[i]} then {sample_vars[i+1]}. '
        output += generation_str
    return [o.strip() for o in output.strip().split('.')][:-1], sample_vars
    
ret = build_logical_statement(3, uppercases)
ret

(['If not D then H', 'If H then not B'], ['not D', 'H', 'not B'])

# Drafting dataset generation method

### Basic dataset with pre-defined, fixed facts.

In [4]:
# With numpy array, make a 3d matrix showing beliefs of beliefs
level_two_bg = np.random.rand(3,3,3).round()
level_two_bg

array([[[1., 1., 1.],
        [0., 1., 0.],
        [1., 0., 0.]],

       [[0., 1., 0.],
        [0., 0., 1.],
        [1., 0., 1.]],

       [[1., 1., 1.],
        [1., 0., 1.],
        [1., 0., 0.]]])

In [5]:
# n_people x n_facts x n_people matrix where for each person i, M[i,i] are there own beliefs for every fact, M[i, x] are their beleifs about person x
level_two_bg[0,0]

array([1., 1., 1.])

In [6]:
num_people = 5
num_facts = 4
people = ['Alice', 'Bob', 'Joe', 'Carlos', 'Jane']
facts = ['wumpuses are jumpuses', 'bombos have zombos', 'chowkey need joobers','dooberts have yompers','roules have gravems']

info_arr = np.random.rand(num_people,num_people,num_facts).round()
print(info_arr)

[[[1. 1. 1. 1.]
  [0. 1. 0. 0.]
  [0. 1. 0. 1.]
  [1. 0. 0. 0.]
  [1. 0. 0. 0.]]

 [[0. 1. 0. 0.]
  [1. 0. 1. 1.]
  [0. 0. 1. 0.]
  [0. 0. 0. 0.]
  [0. 1. 0. 0.]]

 [[0. 1. 0. 0.]
  [1. 0. 1. 1.]
  [1. 1. 1. 0.]
  [0. 0. 1. 1.]
  [0. 0. 1. 0.]]

 [[0. 0. 1. 1.]
  [0. 1. 1. 1.]
  [1. 0. 1. 0.]
  [0. 0. 0. 0.]
  [0. 1. 1. 0.]]

 [[0. 1. 0. 0.]
  [1. 1. 0. 0.]
  [1. 1. 0. 0.]
  [0. 1. 1. 1.]
  [1. 0. 1. 1.]]]


In [7]:
dataset = []

# dimensions: num_people x num_people x num_facts
for i in range(info_arr.shape[0]):
    subject = people[i]
    #output = f'{subject} '
    output = ''
    for j in range(info_arr.shape[1]):
        if i == j:
            # your own beliefs
            output += f'{subject} '
        else:
            # other's beliefs
            output += f'{subject} thinks {people[j]} '
        for k in range(info_arr.shape[2]):
            thought = 'believes' if info_arr[i,j,k] == 1 else 'does not believe'
            if k == info_arr.shape[2] - 1:
                output += f'and {thought} {facts[k]}. '
            else:
                output += f'{thought} {facts[k]}, '
    print(output + '\n')
    
    

Alice believes wumpuses are jumpuses, believes bombos have zombos, believes chowkey need joobers, and believes dooberts have yompers. Alice thinks Bob does not believe wumpuses are jumpuses, believes bombos have zombos, does not believe chowkey need joobers, and does not believe dooberts have yompers. Alice thinks Joe does not believe wumpuses are jumpuses, believes bombos have zombos, does not believe chowkey need joobers, and believes dooberts have yompers. Alice thinks Carlos believes wumpuses are jumpuses, does not believe bombos have zombos, does not believe chowkey need joobers, and does not believe dooberts have yompers. Alice thinks Jane believes wumpuses are jumpuses, does not believe bombos have zombos, does not believe chowkey need joobers, and does not believe dooberts have yompers. 

Bob thinks Alice does not believe wumpuses are jumpuses, believes bombos have zombos, does not believe chowkey need joobers, and does not believe dooberts have yompers. Bob believes wumpuses a

Letters can be replaced with facts about the world. Could also be replaced with relations between other people.

This setup above could lead to asking question like:
- Do Bob and Alice agree about A? *Answer is checking if M[0,0,0] == M[1,1,0]*
- Assume A implies doing X. Would Alice do X? Would Bob be surpised to see Alice doing X? *Answers would be just checking if M[1,0,0] == 1* 
- Alice calls a meeting with everyone she knows that disagrees with her on B. Would Joe be at this meeting? *Answer is checking if M[2,1,2] == M[0,0,1]*


Alice believes wumpuses are jumpuses, believes bombos have zombos, does not believe chowkey need joobers, and believes dooberts have yompers. Alice thinks Bob does not believe wumpuses are jumpuses, believes bombos have zombos, does not believe chowkey need joobers, and believes dooberts have yompers. Alice thinks Joe believes wumpuses are jumpuses, believes bombos have zombos, believes chowkey need joobers, and believes dooberts have yompers. Alice thinks Carlos does not believe wumpuses are jumpuses, does not believe bombos have zombos, believes chowkey need joobers, and believes dooberts have yompers. Alice thinks Jane does not believe wumpuses are jumpuses, believes bombos have zombos, does not believe chowkey need joobers, and does not believe dooberts have yompers. 

Bob thinks Alice does not believe wumpuses are jumpuses, does not believe bombos have zombos, does not believe chowkey need joobers, and does not believe dooberts have yompers. Bob believes wumpuses are jumpuses, believes bombos have zombos, believes chowkey need joobers, and believes dooberts have yompers. Bob thinks Joe does not believe wumpuses are jumpuses, does not believe bombos have zombos, does not believe chowkey need joobers, and believes dooberts have yompers. Bob thinks Carlos does not believe wumpuses are jumpuses, believes bombos have zombos, believes chowkey need joobers, and believes dooberts have yompers. Bob thinks Jane does not believe wumpuses are jumpuses, believes bombos have zombos, does not believe chowkey need joobers, and believes dooberts have yompers. 

Joe thinks Alice believes wumpuses are jumpuses, does not believe bombos have zombos, believes chowkey need joobers, and does not believe dooberts have yompers. Joe thinks Bob does not believe wumpuses are jumpuses, does not believe bombos have zombos, believes chowkey need joobers, and believes dooberts have yompers. Joe does not believe wumpuses are jumpuses, does not believe bombos have zombos, does not believe chowkey need joobers, and believes dooberts have yompers. Joe thinks Carlos does not believe wumpuses are jumpuses, believes bombos have zombos, does not believe chowkey need joobers, and believes dooberts have yompers. Joe thinks Jane believes wumpuses are jumpuses, believes bombos have zombos, believes chowkey need joobers, and does not believe dooberts have yompers. 

Carlos thinks Alice believes wumpuses are jumpuses, does not believe bombos have zombos, does not believe chowkey need joobers, and does not believe dooberts have yompers. Carlos thinks Bob believes wumpuses are jumpuses, does not believe bombos have zombos, believes chowkey need joobers, and believes dooberts have yompers. Carlos thinks Joe does not believe wumpuses are jumpuses, believes bombos have zombos, does not believe chowkey need joobers, and does not believe dooberts have yompers. Carlos believes wumpuses are jumpuses, believes bombos have zombos, does not believe chowkey need joobers, and believes dooberts have yompers. Carlos thinks Jane does not believe wumpuses are jumpuses, does not believe bombos have zombos, believes chowkey need joobers, and does not believe dooberts have yompers. 

Jane thinks Alice does not believe wumpuses are jumpuses, does not believe bombos have zombos, believes chowkey need joobers, and believes dooberts have yompers. Jane thinks Bob does not believe wumpuses are jumpuses, believes bombos have zombos, does not believe chowkey need joobers, and does not believe dooberts have yompers. Jane thinks Joe does not believe wumpuses are jumpuses, believes bombos have zombos, does not believe chowkey need joobers, and does not believe dooberts have yompers. Jane thinks Carlos believes wumpuses are jumpuses, does not believe bombos have zombos, believes chowkey need joobers, and does not believe dooberts have yompers. Jane does not believe wumpuses are jumpuses, does not believe bombos have zombos, believes chowkey need joobers, and believes dooberts have yompers. 

Do Bob and Alice agree about wumpuses?


### Higher order facts

Notes from Saporov: Let each person be named 1, ..., n. They have their own knowledge graph G_i (this just has the information that person i knows). They also have knowledge graphs G_i,j for every other person j which represents what person i believes person j knows. So this generalizes your setup a bit in that it's not all or nothing: a person can know some parts of what another person believes but not necessarily everything (or nothing).

In theory, you could go even further and say for every (i,j), there is a graph for every other person k, call it G_i,j,k that represents what person i believes person j believes person k knows. And so on.

Another way to look at it could be to create a new kind of "higher-order" fact like `believes(sally,likes(alex,jay))` which would have a meaning like "Sally thinks that Alex likes Jay." And instead of having a hierarchy of multiple knowledge graphs, you can just have one knowledge graph per person G_i, except the knowledge graphs can contain facts like `believes(i,e)`. Note that you can have nested logical forms like `believes(sally,believes(jen,likes(alex,jay)))` which would mean "Sally thinks that Jen thinks that Alex likes Jay." This language would have good expressive power since we can make distinctions like: believes(sally,~likes(alex,jay)) which would mean "Sally believes that Alex does not like Jay," vs ~believes(sally,likes(alex,jay)) which would mean "Sally does not believe that Alex likes Jay" (notice that the first implies the second but the second doesn't necessarily imply the first). So we can have a rule of inference like: for all i and e, if believes(i,~e) then ~believes(i,e).


In [8]:
# First create a knowledge graph that people's beliefs are based on
num_people = 3
people = ['Alice', 'Bob', 'Joe']

In [9]:
world_relation_graph = np.random.rand(num_people,num_people).round()
print(world_relation_graph)

[[0. 1. 1.]
 [1. 0. 1.]
 [1. 0. 1.]]


In [10]:
# Translate to string
for i in range(world_relation_graph.shape[0]):
    output = ''
    for j in range(world_relation_graph.shape[1]):
        if i != j:
            output += f'{people[i]}'
            relation = 'likes' if world_relation_graph[i,j] == 1 else "doesn't like"
            output += f' {relation} {people[j]}. '
    print(output)

Alice likes Bob. Alice likes Joe. 
Bob likes Alice. Bob likes Joe. 
Joe likes Alice. Joe doesn't like Bob. 


In each persons graph, row i,j singifies agreement with the information given by this matrix.

In [11]:
personal_belief_graph = np.random.rand(num_people,num_people,num_people).round()
print(personal_belief_graph)
print('---')
print(world_relation_graph)

[[[1. 0. 0.]
  [1. 1. 1.]
  [1. 0. 1.]]

 [[0. 0. 0.]
  [1. 0. 1.]
  [0. 1. 0.]]

 [[1. 1. 1.]
  [1. 0. 0.]
  [1. 1. 1.]]]
---
[[0. 1. 1.]
 [1. 0. 1.]
 [1. 0. 1.]]


In [12]:
# Fix each row so that their personal graph equals the world graph on themselves
for i in range(personal_belief_graph.shape[0]):
    for j in range(personal_belief_graph.shape[1]):
        if i == j:
            personal_belief_graph[i,j] = world_relation_graph[i]
print(personal_belief_graph)


[[[0. 1. 1.]
  [1. 1. 1.]
  [1. 0. 1.]]

 [[0. 0. 0.]
  [1. 0. 1.]
  [0. 1. 0.]]

 [[1. 1. 1.]
  [1. 0. 0.]
  [1. 0. 1.]]]


In [13]:
# Turn this into words
for i in range(personal_belief_graph.shape[0]): #In i's view
    
    subject = people[i]
    
    for j in range(personal_belief_graph.shape[1]): 
        if i == j:
            continue            
        for k in range(personal_belief_graph.shape[2]): #j feels x towards k
            if k != j:
                output = f'In {subject}\'s  view, '
                output += f'{people[j]} '
                relation = 'likes' if personal_belief_graph[i, j, k] == 1 else "doesn't like"
                output += f' {relation} {people[k]}.'
                #output += '. '
                print(output)

In Alice's  view, Bob  likes Alice.
In Alice's  view, Bob  likes Joe.
In Alice's  view, Joe  likes Alice.
In Alice's  view, Joe  doesn't like Bob.
In Bob's  view, Alice  doesn't like Bob.
In Bob's  view, Alice  doesn't like Joe.
In Bob's  view, Joe  doesn't like Alice.
In Bob's  view, Joe  likes Bob.
In Joe's  view, Alice  likes Bob.
In Joe's  view, Alice  likes Joe.
In Joe's  view, Bob  likes Alice.
In Joe's  view, Bob  doesn't like Joe.


person sees object in container. person leaves. other person moves object to different container. 1st person comes back. where does person look? would they be surprised?

state 1, everybody has observed it -> state 2, partially observed -> who thinks theyre in state 1 and who thinks theyre in state 2

can false belief be generalized to more people?

world modeling + tom vs world modeling
door problem - num people and rooms vs num movements

Now we can ask questions about whether or not people believe what's in the world info. We can also ask questions with one degree of removal. Being limited to one degree of removal might be bad though.